In [5]:
import torch
import torch.nn as nn

a = torch.rand((16,3,244,244))
print(a)

tensor([[[[0.3411, 0.4671, 0.9660,  ..., 0.0832, 0.2835, 0.7510],
          [0.6007, 0.3166, 0.8528,  ..., 0.1998, 0.9680, 0.3628],
          [0.6893, 0.4726, 0.2534,  ..., 0.6766, 0.3793, 0.2380],
          ...,
          [0.9900, 0.6195, 0.0368,  ..., 0.3988, 0.9780, 0.7392],
          [0.8052, 0.7150, 0.6802,  ..., 0.6190, 0.9863, 0.4066],
          [0.7198, 0.0141, 0.1157,  ..., 0.4635, 0.8063, 0.0450]],

         [[0.6886, 0.6758, 0.6505,  ..., 0.5760, 0.1815, 0.5982],
          [0.6219, 0.4241, 0.3280,  ..., 0.8434, 0.1935, 0.9195],
          [0.8038, 0.8023, 0.2537,  ..., 0.6203, 0.7824, 0.8152],
          ...,
          [0.1070, 0.7794, 0.3275,  ..., 0.3014, 0.4430, 0.7010],
          [0.3961, 0.0116, 0.5504,  ..., 0.4543, 0.2618, 0.3266],
          [0.9592, 0.5611, 0.4911,  ..., 0.0506, 0.5754, 0.9170]],

         [[0.0375, 0.7288, 0.5285,  ..., 0.2156, 0.1032, 0.0818],
          [0.8347, 0.3554, 0.9303,  ..., 0.1207, 0.1049, 0.2514],
          [0.6746, 0.0053, 0.4939,  ..., 0

In [34]:
class Residual_Block(nn.Module):
    def __init__(self, in_dim, mid_dim, out_dim): 
        super().__init__()
        # Residual Block
        self.residual_block = nn.Sequential(
            nn.Conv2d(in_dim, mid_dim, kernel_size=3, padding=1), 
            nn.ReLU(), 
            nn.Conv2d(mid_dim, out_dim, kernel_size=3, padding=1), 
        ) 
        self.relu = nn.ReLU() 
    
    def forward(self, x): 
        print(x.shape)
        out = self.residual_block(x) # F(x) 
        print(x.shape)
        out = out + x # F(x) + x 
        print(x.shape)
        out = self.relu(out)
        print(x.shape)
        return out
        


In [35]:
test = Residual_Block(3,64,128)

In [36]:
test

Residual_Block(
  (residual_block): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (relu): ReLU()
)

In [37]:
a = torch.rand((64,3,3,3))
a

tensor([[[[0.2720, 0.6528, 0.5642],
          [0.6499, 0.7743, 0.1811],
          [0.1142, 0.2996, 0.0201]],

         [[0.2760, 0.0553, 0.6946],
          [0.0301, 0.9812, 0.9633],
          [0.6406, 0.7506, 0.6128]],

         [[0.8578, 0.0531, 0.6006],
          [0.6863, 0.4477, 0.1510],
          [0.6098, 0.4046, 0.2132]]],


        [[[0.6577, 0.6913, 0.1092],
          [0.0139, 0.5946, 0.1769],
          [0.9091, 0.5694, 0.4445]],

         [[0.2440, 0.7253, 0.1915],
          [0.9882, 0.9610, 0.6676],
          [0.0390, 0.9781, 0.9314]],

         [[0.7664, 0.3185, 0.2666],
          [0.4927, 0.1196, 0.8026],
          [0.0067, 0.4388, 0.8912]]],


        [[[0.1636, 0.3616, 0.6946],
          [0.5452, 0.2895, 0.7776],
          [0.9876, 0.4012, 0.1272]],

         [[0.1437, 0.9141, 0.8964],
          [0.2839, 0.1634, 0.1889],
          [0.5121, 0.7645, 0.1984]],

         [[0.1751, 0.2745, 0.2670],
          [0.4053, 0.5207, 0.0599],
          [0.8363, 0.8787, 0.5962]]],


    

In [38]:
test(a)

torch.Size([64, 3, 3, 3])
torch.Size([64, 3, 3, 3])


RuntimeError: The size of tensor a (128) must match the size of tensor b (3) at non-singleton dimension 1